In [1]:
import json
import os
import re
from langdetect import detect
from langdetect import detect_langs
from tqdm import tqdm
import uuid

In [2]:
def detect_language(content):
    #print("context",content)
    lang = detect(content)
    if lang == "zh-cn":
        return "zh"
    if lang == "en":
        return "en"
    return "None"

In [3]:
pattern_list = [r'丁香医生审稿专业委员会同行评议通过\n\n',
                r'(词条作者|审核专家)\n\n([^\n]+)\n\n',
                r'(发布时间|最后修订时间)([^\n]+)\n\n',
                r'((?:\n\n.{2,4})+\n\n更多)',
                r'参考资料\n[\s\S]*',
                r'<sup>.*?</sup>',
                '\n    \n\*   就诊$',
                #下面为新加内容
                
                '( ?图 ? ?\d\，)?图片来源：?([^\n]+?\n)?',
                '主要的表现如下图：',
                '(https?://(www)?[^ ]*)',
                '(网站：)?www\..*\.org/?',
                '嫣然天使基金：400-810-2727，|中国微笑行动：0571-85860800 或 13968175733|美国微笑列车基金：座机拨打 800-810-0059，|李嘉诚重生行动：010-63584979 或 010-63584975 或 010-83539988-6025/6040','（见下图）','（如下图位置）','（?图源.*\n','\*\*.*?高能预警 ?↓?\*\*','高能预警↓','来源：张建明《皮肤性病诊断图谱》',r' 多余毛发的去除 UpToDate','（如?下?图 ?[\d\w]?）','（?图 ?\d ?来源：自?UpToDate ?临床顾问）?','\n    图 \d 新生[^来]*来源：UpToDate\n','来源：张建明《皮肤性病诊断图谱》','Gerald W Smetana, MD. Evaluation of the patient with night sweats or generalized hyperhidrosis\.|Hung K So1, Albert M Li1, Chun T Au1, etc. Night sweats in children: prevalence and associated factors.','Noreen Iftikhar. Why Is My Child Sweating at Night and What Can I Do\?','Jennifer Shu. Baby and Child Health: The Essential Guide From Birth to 11 Years.','\\\\\[\d\\\]',
                                '（图 ? ?\d ?）','图 \d.*（引自参考文献 ）',"[\(（]*?图 [\d\w].*来源：?(于(编辑)?(网络)?)?(参考文献)?(于)?(文献)?(Andrews' Diseases of the Skin Clinical Dermatology）)?(《中国临床皮肤病学》)?(作者临床搜集）)?(Dermatology)?",'图 \d：.*（图片来自网络）','图\w. (子宫肌纤维在子宫复旧过程中由长变短)?(产后 5 天的子宫示意图)?','[（(具体)见如]*[^。，\n（拍]*下图[^。，\n）可像就顺小(中眼球)]+）?(的后面)?。?，?；?','[（(具体)见如]*下图。?',
                '[（如见下方]*?(（舔唇性皮炎，)?(（幼年型跖部皮肤病，)?图 \d[^\n，；），]*[），]?[(脑组织下疝)]*(基本都也在 22～26 个月内闭合；)?(没有风团)?(部分有结痂)?',
                '点击上方图片可以预约流感疫苗接种|具体可见上文图片。|具体可以参考下面这张图。|图片：胎儿.位|（如下面左图）|图片：作者提供|如上图，|具体内容可以查看下面图片：|\*\*特别提醒：\*\* 图片仅为个案展示，不能作为诊断依据。如果怀疑得了疾病，还请尽早在线问诊或去线下医院寻求诊疗帮助。(可以在本 App 里搜索「丘疹性荨麻疹」或者「皮肤科」咨询相关的医生。)?|张建明、李海英《皮肤性病诊断图谱》|（图1，照片来源：参考资料  ）|（图片 \d）|图片：丁香妈妈（自己拍的）|图\d\. .癣：...|图1.正常肚脐的位置和形态|图2.干结的脐带残端|图片参考：见水印|（具体可参考图片）',
                '（来源：教科书）|[(数据)(表格)(表 1 资料)（]+来源：[(WHO)(屈腕试验（Phalen 征）\-国内版 Bing images)(Froment 征\\_百度百科 \(baidu.com\))(参考文献)(教科书）)(中国食物成分表 第六版 第二册)(儿科学第九版)(付超 《正常男性阴茎发育调查》))]+|[表格数据]+来源：'

                
                

]


In [63]:
def clean_text(context):
    for pattern in pattern_list:
        #print(re.findall(pattern, context),context)
        context = re.sub(pattern,'',context)
    
    # 消除空行问题
    context = re.sub(r'\n +\n',"\n\n",context)
    context = re.sub(r'\n +\n',"\n\n",context)

    # print({"text":context})
    context=re.sub('[^\n]\n    --',"\n\n    --",context)
    context = re.sub("\n{2,}","\n\n",context)
    split_token = "\n\n"
    new_context = []
    context = context.split(split_token)
    for item in context:
        if "* " not in item:
            item = item.strip(" ")
        else:
            pass
        new_context.append(item)
    
    return split_token.join(new_context)


In [65]:
fw = open("dingxiangyisheng_test.jsonl","w",encoding="utf-8")
with open("../../../../full_data/dingxiangyisheng/dingxiangyisheng.jsonl","r",encoding="utf-8") as fs:
    for items in tqdm(fs.readlines()):
        item = json.loads(items.strip())
        context = item["text"]
        # if "胸腔积液的患者多吗？" not in context:
        #     continue
        lang = detect_language(context)
        context = clean_text(context)
        context = context.strip(" ").strip("\n").strip(" ").strip("\n")
        # print({"text":context})
        uid = str(uuid.uuid4())
        #print(context)
        new_data = {
            "seq_id":uid,
            "title":item["title"],
            "text":context,
            "tags":{},
            "lang":lang,
            "attr":{
                "category":[item["category"]],
                "src_url":item["src_url"],
            },
            "ext":None,
            "dataset":"dingxiangyisheng",
            "batch_name":"20240312",
            "version":"version0",
        }
        new_data = json.dumps(new_data,ensure_ascii=False)
        fw.write(new_data+"\n")

        #break

100%|██████████| 5582/5582 [00:45<00:00, 123.81it/s]
